In [ ]:
# PS4 - CE264
# GSI: Mustapha Harb, Mengqiao Yu, Andrew Campbell

# importing the requried libraries
from collections import OrderedDict    # For recording the model specification 

import pandas as pd                    # For file input/output
import numpy as np                     # For vectorized math operations

import pylogit as pl                   # For MNL model estimation and
                                       # conversion from wide to long format

# reading the data file 
data_wide  = pd.read_csv("data01.csv",sep=",")

In [ ]:
# converting the data from wide to long format

# Create the list of individual specific variables
ind_variables = data_wide.columns.tolist()[:2] + ["weights"]

# Specify the variables that vary across individuals and some or all alternatives
# The keys are the column names that will be used in the long format dataframe.
# The values are dictionaries whose key-value pairs are the alternative id and
# the column name of the corresponding column that encodes that variable for
# the given alternative. Examples below.
alt_varying_variables = {u'travel_time': dict([(1, 'tt_da'),
                                               (2, 'tt_sr'),
                                               (3, 'tt_walk'),
                                               (4, 'tt_bike'),
                                               (5, 'tt_wt'),
                                               (6, 'tt_dt')]),
                          u'distance_car': dict([(1, 'dist_car'),
                                                (2, 'dist_car')]),
                          u'travel_cost': dict([(1, 'cost_da'),
                                                (2, 'cost_sr'),
                                                (5, 'cost_wt'),
                                                (6, 'cost_dt')]),
                          u'access_time': dict([(5, 'accTime_wt'),
                                                (6, 'accTime_dt')]),
                          u'egress_time': dict([(5, 'egrTime_wt'),
                                                (6, 'egrTime_dt')]),
                          u'initial_wait': dict([(5, 'iWait_wt'),
                                                 (6, 'iWait_dt')]),
                          u'transfer_wait': dict([(5, 'xWait_wt'),
                                                  (6, 'xWait_dt')]),
                          u'access_distance_dt': dict([(6, "accDist_dt")])}

# Specify the availability variables
# Note that the keys of the dictionary are the alternative id's.
# The values are the columns denoting the availability for the
# given mode in the dataset.


availability_variables = {1: 'avail_da',
                          2: 'avail_sr', 
                          3: 'avail_walk',
                          4: 'avail_bike',
                          5: 'avail_wt',
                          6: 'avail_dt'}

##########
# Determine the columns for: alternative ids, the observation ids and the choice
##########
# The 'custom_alt_id' is the name of a column to be created in the long-format data
# It will identify the alternative associated with each row.
custom_alt_id = "mode_id"

# Create a custom id column that ignores the fact that this is a 
# panel/repeated-observations dataset. Note the +1 ensures the id's start at one.
obs_id_column = "obsID"

# Create a variable recording the choice column
choice_column = "choice"

In [ ]:
# Perform the conversion to long-format
data_long = pl.convert_wide_to_long(data_wide, 
                                           ind_variables, 
                                           alt_varying_variables, 
                                           availability_variables, 
                                           obs_id_column, 
                                           choice_column,
                                           new_alt_id_name=custom_alt_id)
# Look at the resulting long-format dataframe
data_long.head(10).T

In [ ]:
##########
# Create scaled variables so the estimated coefficients are of similar magnitudes
##########
# Scale the travel time column by 60 to convert raw units (minutes) to hours
data_long["travel_time_hrs"] = data_long["travel_time"] / 60.0

# Scale the access by 60 to convert raw units (minutes) to hours
data_long["access_time_hrs"] = data_long["access_time"] / 60.0

# for drive to transit let us combine travel time and access time
data_long["travel_time_access_time_hrs"] = data_long["travel_time_hrs"] + data_long["access_time_hrs"]

#Scale the egress time by 60
data_long["egress_time_hrs"] = data_long["egress_time"] / 60.0

# combining access and egress time which we want to use for the walk to transit alternative
data_long["acess_egress_hrs"] = data_long["access_time_hrs"] + data_long["egress_time_hrs"]

# scaling the initial wait by 60
data_long["initial_wait_hrs"] = data_long["initial_wait"] / 60.0

# scaling the transfer wait by 60
data_long["transfer_wait_hrs"]  = data_long["transfer_wait"] / 60.0

# combining transfer wait and initial wait to be used for walk to transit and bike to transit
data_long["initial_transfer_wait_hrs"] = data_long["initial_wait_hrs"] + data_long["transfer_wait_hrs"]


# creating non-linear transformations for the cost variable
cutOff1 = 2
cutOff2 = 7


data_long["cost_cat_one"] = (data_long["travel_cost"] <= cutOff1)*data_long["travel_cost"] + (data_long["travel_cost"] > cutOff1)*cutOff1

data_long["cost_cat_two"] = (data_long["travel_cost"] > cutOff1)*(data_long["travel_cost"] <= cutOff2)*(data_long["travel_cost"] - cutOff1) + (data_long["travel_cost"] > cutOff2)* (cutOff2 - cutOff1)

data_long["cost_cat_three"] = (data_long["travel_cost"] > cutOff2)*(data_long["travel_cost"] - cutOff2)



# specifying the utility equations

# NOTE: - Specification and variable names must be ordered dictionaries.
#       - Keys should be variables within the long format dataframe.
#         The sole exception to this is the "intercept" key.
#       - For the specification dictionary, the values should be lists
#         of integers or or lists of lists of integers. Within a list, 
#         or within the inner-most list, the integers should be the 
#         alternative ID's of the alternative whose utility specification 
#         the explanatory variable is entering. Lists of lists denote 
#         alternatives that will share a common coefficient for the variable
#         in question.

basic_specification = OrderedDict()
basic_names = OrderedDict()


basic_specification["intercept"] = [ 2, 3, 4, 5, 6]
basic_names["intercept"] = ['ASC SR',
                            'ASC Walk', 'ASC Bike', 'ASC WT', 'ASC DT']

basic_specification["travel_time_hrs"] = [[1, 2, 5], 4, 3]
basic_names["travel_time_hrs"] = ['In-Vehicle Travel Time, units:hrs (DA, SR, WT)',
                                  'Bike Time, units:hrs (Bike)',
                                  'Walk Time, units:hrs (Walk)']

basic_specification["travel_time_access_time_hrs"] = [6]
basic_names["travel_time_access_time_hrs"] = ["In-Vehicle Travel Time, units:hrs, (DT)"]

basic_specification["acess_egress_hrs"] = [5]
basic_names["acess_egress_hrs"] = ["Walk Time, units:hrs, (WT)"]

basic_specification["egress_time_hrs"] = [6]
basic_names["egress_time_hrs"] = ["Walk Time, units:hrs, (DT)"]

basic_specification["initial_transfer_wait_hrs"] = [[5, 6]]
basic_names["initial_transfer_wait_hrs"] = ["Waiting Time, units:hrs, (WT and DT)"]


basic_specification["cost_cat_one"] = [[1, 2, 5,6]]
basic_names["cost_cat_one"] = ['Cost: Under $2']

basic_specification["cost_cat_two"] = [[1, 2, 5,6]]
basic_names["cost_cat_two"] = ['Cost: (2 - 7)$']

basic_specification["cost_cat_three"] = [[1, 2, 5,6]]
basic_names["cost_cat_three"] = ['Cost: Above $7']



In [ ]:
# taking a sample of 10,000 observation from the BATS 2000 dataset
new_data = data_long.loc[data_long[obs_id_column].isin(range(10000))].copy()

In [ ]:
# Estimate the multinomial logit model (MNL)
data_mnl = pl.create_choice_model(data=new_data,
                                  alt_id_col=custom_alt_id,
                                  obs_id_col=obs_id_column,
                                  choice_col=choice_column,
                                  specification=basic_specification,
                                  model_type="MNL",
                                  names=basic_names)

# Specify the initial values and method for the optimization.
data_mnl.fit_mle(np.zeros(15))



In [ ]:
# Look at the estimation results
data_mnl.get_statsmodels_summary()

In [ ]:
# prediction - sample enumeration
# array of probabilities for each available alternative for all individuals in the new_data file that 
# was used for estimation
prediction_array = data_mnl.predict(new_data)

In [ ]:
# check out the probabilities
prediction_array

In [ ]:
# market shares
# the following script performs sample enumeration for all alternatives by catering for individual weights 

total_weights = np.sum(data_wide.loc[:9999, 'weights'])
weights = new_data["weights"] / total_weights

alternative_names = ["drive", "shared ride", "walk", "bike",
                     "walk transit", "drive transit"]
num_alternatives = data_long["mode_id"].unique().size
market_shares = np.empty(num_alternatives)
for i in range(1, 7):
    filter_condition = (new_data["mode_id"] == i).values
    num_obs_in_condition = filter_condition.sum()
    current_weights = weights[filter_condition]
    assert current_weights.size == num_obs_in_condition
    current_alternative_share =\
        prediction_array[filter_condition].dot(current_weights)
    market_shares[i - 1] = current_alternative_share
    
pd.Series(market_shares, index=alternative_names)